In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 80 kB/s 


In [ ]:
import datetime
import pickle
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/날씨')
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import itertools

import pandas as pd
import pandas_profiling
import numpy as np
import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid' )
#import scikitplot

from scipy import stats


import sklearn
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import metrics
from sklearn.utils.fixes import loguniform
from sklearn import model_selection
from sklearn import ensemble
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
import xgboost as xgb
import lightgbm as lgb 
import xgboost as xgb
import catboost as cb
import statsmodels.api as sm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_test = pd.read_pickle("/content/drive/MyDrive/날씨/lgb_test.pkl")
df_test.drop('uv',axis=1,inplace=True)

In [ ]:
df_train = pd.read_pickle("/content/drive/MyDrive/날씨/lgb_train.pkl")
df_train

,uv,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,year,Month sin,Month cos,Day sin,Day cos,2/Day sin,2/Day cos,3/Day sin,3/Day cos,stn
date_time,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,0.0,0.009771,0.009774,0.011324,0.010648,0.019778,0.018657,0.536633,0.680527,0.747787,...,0.0,0.487099,0.999834,0.500000,1.000000,0.500000,1.000000,0.500000,1.000000,131
2020-01-01 00:10:00,0.0,0.009279,0.009774,0.010194,0.011053,0.018288,0.017717,0.534080,0.677492,0.745142,...,0.0,0.487816,0.999852,0.521810,0.999524,0.543578,0.998097,0.565263,0.995722,131
2020-01-01 00:20:00,0.0,0.009279,0.009286,0.011324,0.010378,0.019778,0.018657,0.535360,0.675969,0.743373,...,0.0,0.488532,0.999868,0.543578,0.998097,0.586824,0.992404,0.629410,0.982963,131
2020-01-01 00:30:00,0.0,0.009279,0.009286,0.012740,0.010243,0.020269,0.018657,0.513766,0.679011,0.743816,...,0.0,0.489249,0.999884,0.565263,0.995722,0.629410,0.982963,0.691342,0.961940,131
2020-01-01 00:40:00,0.0,0.008796,0.009286,0.011324,0.010648,0.019287,0.017717,0.509430,0.679011,0.744700,...,0.0,0.489966,0.999899,0.586824,0.992404,0.671010,0.969846,0.750000,0.933013,131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:10:00,0.0,0.009279,0.010963,0.011709,0.011562,0.020565,0.017734,0.725256,0.573162,0.628261,...,1.0,0.536638,0.998656,0.391780,0.988148,0.288691,0.953154,0.195619,0.896677,159
2021-12-31 23:20:00,0.0,0.009771,0.010416,0.012314,0.009630,0.021640,0.018675,0.723271,0.567629,0.616301,...,1.0,0.537353,0.998603,0.413176,0.992404,0.328990,0.969846,0.250000,0.933013,159
2021-12-31 23:30:00,0.0,0.009771,0.010963,0.011709,0.009906,0.021103,0.018675,0.721257,0.569480,0.617876,...,1.0,0.538067,0.998549,0.434737,0.995722,0.370590,0.982963,0.308658,0.961940,159


In [ ]:
def  stn_split(df:pd.DataFrame,spn_num :int)->pd.DataFrame:
  if 'stn' in df.columns:
    df_stn_num = df.loc[df['stn']==spn_num,:]
    df_stn_num = df_stn_num.drop(['stn'],axis=1)
    return df_stn_num

def time_feature(df:pd.DataFrame)->pd.DataFrame:
  x_list = ['solarza','esr','band1','band2','band3','band4','band6','band7','band12']
  df_feature = df.copy()
  for x in x_list:
    for n in [1,144]:
      if f'{x}_{n}' not in df_feature.columns:
        df_feature[f'{x}_{n}'] = df_feature[x].diff(n).bfill()
    for n in [72,144]:
      if f'{x}_{n}ma' not in df_feature.columns:
        df_feature[f'{x}_{n}ma'] = df_feature[x].rolling(n).mean().bfill()
  return df_feature    

In [ ]:
# sklearn random,grid_search
lgbr=  lgb.LGBMRegressor(colsample_bytree=1,
                         learning_rate = 0.09,
                         max_depth=15,
                         min_child_samples=39,
                         n_estimators=1200,
                         num_leaves=78,
                         subsample=0.5,
                         subsample_feq=1,
                         reg_alpha =0.1,
        
                         n_jobs=-1,
                         randoms_state=42)

In [ ]:
#optuna
lgb_parameters = {'n_estimators': 1181,
                  'max_depth': 21, 
                  'colsample_bytree': 0.9900654712519903,
                  'learning_rate': 0.09205453627834828,
                  'min_child_samples': 44, 
                  'min_split_gain': 3.171414772101719e-05, 
                  'num_leaves': 75, 'reg_alpha': 1.3736321906283118, 
                  'reg_lambda': 0.00019783308128221966, 
                  'subsample': 0.9100767985669579, 'subsample_freq': 1}


lgbr=  lgb.LGBMRegressor(**lgb_parameters,n_jobs=-1,randomstate=42)


In [ ]:
knnr = KNeighborsRegressor(p=1,
                           algorithm='ball_tree',
                           leaf_size=28,
                           n_neighbors=3,
                           n_jobs=-1)

In [ ]:
xgbr_params = {
  'colsample_bylevel': 0.65,
 'colsample_bynode': 0.7,
 'colsample_bytree': 0.9,
 'gamma': 0.1179063358441864,
 'grow_policy': 'depthwise',
 'learning_rate': 0.08834615853128412,
 'max_bin': 75,
 'max_depth': 19,
 'max_leaves': 13,
 'min_child_weight': 9,
 'n_estimators': 873,
 'nomarlize_type': 'tree',
 'one_drop': 1,
 'rate_drop': 0.689790493226518,
 'reg_alpha': 0,
 'reg_lambda': 3,
 'sample_type': 'weighted',
 'skip_drop': 0.34570580184187344,
 'subsample': 0.8}
 
xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state=42,
                         **xgbr_params)

In [ ]:
etr_parameters =  {'n_estimators': 398,
                       'max_depth': 25, 
                       'min_weight_fraction_leaf': 7.412742668612509e-05, 
                       'min_samples_split': 0.00035665374622706513, 
                       'min_samples_leaf': 1, 
                       'max_features': 0.8203707539398322,
                       'min_impurity_decrease': 2.6521473059677293e-05}

etr = ExtraTreesRegressor(random_state=42, n_jobs=-1,**etr_parameters)
              


In [ ]:
ctr_param =  {'grow_policy': 'Depthwise', 
             'learning_rate': 0.022649551723252975, 
              'n_estimators': 2710, 
              'max_depth': 10,
              'random_strength': 81,
              'l2_leaf_reg': 0.6369304397693818,
              'max_bin': 112, 
              'od_type': 'Iter', 
              'colsample_bylevel': 0.75, 
              'bootstrap_type': 'Bernoulli',
              'min_child_samples': 52, 
              'subsample': 0.8}

ctr = cb.CatBoostRegressor(**ctr_param,random_state=42)

In [ ]:
ctr_gpu_param  = {'grow_policy': 'Lossguide', 
                  'learning_rate': 0.060397977516964726, 
                  'n_estimators': 2783, 
                  'max_depth': 14, 
                  'random_strength': 96, 
                  'l2_leaf_reg': 0.02830329067016825, 
                  'max_bin': 182, 'od_type': 'Iter', 
                  'bootstrap_type': 'Bernoulli',
                  'min_child_samples': 41, 
                  'num_leaves': 41,
                  'subsample': 0.7}

ctr = cb.CatBoostRegressor(**ctr_gpu_param,random_state=42,task_type = 'GPU')

In [ ]:
rfr_parameters = {'n_estimators': 474, 
                 'max_depth': 20, 
                 'min_samples_split': 0.0010397369608382374, 
                 'min_samples_leaf': 5, 
                 'max_features': 0.8771571391039096,
                 'min_impurity_decrease': 0.0005334014987293961}

rfr = RandomForestRegressor(**rfr_parameters,random_state=42,n_jobs=-1)

In [ ]:
xgb_summer_params = {'grow_policy': 'lossguide', 'n_estimators': 508, 'max_depth': 30, 'max_leaves': 80, 'learning_rate': 0.14806377331017967, 'gamma': 0.3641261391337236, 'min_child_weight': 9, 'subsample': 0.8911091481390034, 'colsample_bytree': 0.8351212630407904, 'colsample_bylevel': 0.6740549802964637, 'colsample_bynode': 0.9692187477250439, 'reg_lambda': 4, 'reg_alpha': 1, 'max_bin': 137, 'sample_type': 'weighted', 'rate_drop': 0.7377036641002843, 'one_drop': 0, 'skip_drop': 0.27298904790196726}
xgb_summer_params2 =  {'grow_policy': 'lossguide', 'n_estimators': 1570, 'max_depth': 95, 'max_leaves': 76, 'learning_rate': 0.1493922711356767, 'gamma': 0.11079073578898974, 'min_child_weight': 4, 'subsample': 0.8954837522622376, 'colsample_bytree': 0.9243106076919249, 'colsample_bylevel': 0.6686071550993263, 'colsample_bynode': 0.87542298251727, 'reg_lambda': 5, 'reg_alpha': 0, 'max_bin': 135, 'sample_type': 'weighted', 'rate_drop': 0.963902626727559, 'one_drop': 1, 'skip_drop': 0.3880536098345327}


xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state=42,
                         **xgb_summer_params2)


In [ ]:
final_result = []
for stn in set(df_train['stn']):
    result = pd.DataFrame({'YearMonthDayHourMinute':[], 'STN':[],'UV':[]})   
    stn_df = stn_split(df_train,stn)
    stn_test = stn_split(df_test,stn)
    stn_df = time_feature(stn_df)
   # stn_df = stn_df.loc[stn_df.index.month.isin([5,6,7,8]),:]
    stn_test = time_feature(stn_test)
    ctr.fit(stn_df.iloc[:,1:],stn_df['uv']) #change
    result['UV'] = ctr.predict(stn_test)   #chagne
    result['STN'] = stn
    result['YearMonthDayHourMinute'] = stn_test.index
    final_result.append(result)
final_result = pd.concat(final_result)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
566:	learn: 0.2004337	total: 12.8s	remaining: 50s
567:	learn: 0.2002757	total: 12.8s	remaining: 49.9s
568:	learn: 0.2000813	total: 12.8s	remaining: 49.9s
569:	learn: 0.1999304	total: 12.9s	remaining: 49.9s
570:	learn: 0.1998199	total: 12.9s	remaining: 49.9s
571:	learn: 0.1996721	total: 12.9s	remaining: 49.9s
572:	learn: 0.1994731	total: 12.9s	remaining: 49.8s
573:	learn: 0.1993581	total: 12.9s	remaining: 49.8s
574:	learn: 0.1992411	total: 13s	remaining: 49.8s
575:	learn: 0.1991438	total: 13s	remaining: 49.7s
576:	learn: 0.1990046	total: 13s	remaining: 49.7s
577:	learn: 0.1988516	total: 13s	remaining: 49.7s
578:	learn: 0.1987027	total: 13.1s	remaining: 49.7s
579:	learn: 0.1985775	total: 13.1s	remaining: 49.7s
580:	learn: 0.1984166	total: 13.1s	remaining: 49.6s
581:	learn: 0.1982779	total: 13.1s	remaining: 49.6s
582:	learn: 0.1981571	total: 13.1s	remaining: 49.6s
583:	learn: 0.1980319	total: 13.2s	remaining: 49.5s
584:	learn: 0.1978797	total: 13.2s	rem

In [ ]:
final_result.set_index(['YearMonthDayHourMinute', 'STN'], inplace=True)
final_result.sort_index(level='STN', inplace=True)
final_result.reset_index(inplace=True)
final_result.sort_values(by=['YearMonthDayHourMinute', 'STN'], inplace=True)
final_result.reset_index(drop=True, inplace=True)

df_sub = pd.read_csv("/content/drive/MyDrive/날씨/박형준/1-1_검증데이터셋.csv")
df_sub['UV'] = final_result['UV']
df_sub

,YearMonthDayHourMinute,STN,UV
0,202206010000,13,-0.001585
1,202206010000,105,0.001834
2,202206010000,108,0.001164
3,202206010000,112,0.005361
4,202206010000,115,-0.003850
...,...,...,...
51850,202206250000,146,0.088405
51851,202206250000,152,0.012940
51852,202206250000,156,0.000913
51853,202206250000,159,0.002642


In [ ]:
df_sub.to_csv("/content/drive/MyDrive/날씨/형한결/ctr_gpu.csv", index=False, encoding='utf-8') #change name

### VOTE

In [ ]:
import glob
result_csv_list =glob.glob("형한결/voting/*.csv")

In [ ]:
result_csv_list.pop(0)

'형한결/voting/subknnr.csv'

In [ ]:
'asdasdzxc' in 'asd'

False

In [ ]:
result_csv_list

['형한결/voting/xgbdart.csv',
 '형한결/voting/etr.csv',
 '형한결/voting/lgb_optuna.csv',
 '형한결/voting/ctr.csv',
 '형한결/voting/rfr.csv',
 '형한결/voting/ctr_gpu.csv']

In [ ]:
result_list = []
for csv in result_csv_list:
  df = pd.read_csv(csv)
  

  if 'xgb' in csv:
    df['UV']=df['UV']*0.2
  elif 'ctr_gpu' in csv:
    df['UV']=df['UV']*0.25
  elif 'ctr' in csv:
    df['UV']=df['UV']*0.25
  elif 'lgb' in csv:
    df['UV']=df['UV']*0.2
  elif 'etr' in csv:
    df['UV']=df['UV']*0.05
  elif 'rfr' in csv:
    df['UV']=df['UV']*0.05 

  result_list.append(df)

In [ ]:
final = result_list[0]
for i in range(len(result_list)-1):
  final = pd.merge(final,result_list[i+1], on=['YearMonthDayHourMinute','STN'])

In [ ]:
final

,YearMonthDayHourMinute,STN,UV_x,UV_y,UV_x,UV_y,UV_x,UV_y
0,202206010000,13,0.005815,0.000046,-0.000087,0.008181,0.000153,-0.000396
1,202206010000,105,-0.001089,0.000033,-0.000039,0.000443,0.000151,0.000459
2,202206010000,108,-0.001891,0.000026,-0.000069,0.001747,0.000104,0.000291
3,202206010000,112,0.007481,0.000035,-0.000248,0.010225,0.000154,0.001340
4,202206010000,115,-0.005263,0.000041,-0.000650,0.003209,0.000135,-0.000963
...,...,...,...,...,...,...,...,...
51850,202206250000,146,0.013467,0.004473,0.017570,0.017131,0.003600,0.022101
51851,202206250000,152,0.007759,0.000044,-0.000623,0.007085,0.000127,0.003235
51852,202206250000,156,-0.001690,0.000039,-0.000427,0.001593,0.000144,0.000228
51853,202206250000,159,-0.000386,0.000030,0.000034,0.008296,0.000182,0.000661


In [ ]:
voting = pd.DataFrame()

In [ ]:
voting['YearMonthDayHourMinute'] = final['YearMonthDayHourMinute']
voting['STN'] = final['STN']

In [ ]:
voting

,YearMonthDayHourMinute,STN
0,202206010000,13
1,202206010000,105
2,202206010000,108
3,202206010000,112
4,202206010000,115
...,...,...
51850,202206250000,146
51851,202206250000,152
51852,202206250000,156
51853,202206250000,159


In [ ]:
voting['UV'] =  final.loc[:,~final.columns.isin(['YearMonthDayHourMinute','STN'])].sum(axis=1)
#voting['UV'] = voting['UV']/7
voting.to_csv('weighted_voting.csv',index=False, encoding='utf-8') #change

In [ ]:
voting

,YearMonthDayHourMinute,STN,UV
0,202206010000,13,0.013713
1,202206010000,105,-0.000042
2,202206010000,108,0.000208
3,202206010000,112,0.018987
4,202206010000,115,-0.003491
...,...,...,...
51850,202206250000,146,0.078342
51851,202206250000,152,0.017627
51852,202206250000,156,-0.000112
51853,202206250000,159,0.008816
